In [ ]:
! pip install pysrt

In [ ]:
import os
import pysrt
import time
import pandas as pd

In [ ]:
subtitles_path = 'data/urheiluruutu/'

In [ ]:
def to_seconds(str_time):
    str_time = str_time.replace(',', '.')
    if '.' not in str_time: 
        str_time = str_time + '.000'
    assert(time.strptime(str_time,'%H:%M:%S.%f'))
    seconds = sum(x * float(t) for x, t in zip([3600, 60, 1], str_time.split(":")))
    return seconds

In [ ]:
data = []
for filename in os.listdir(subtitles_path):
    print(subtitles_path + filename)
    program_id = filename.split(".mp4_fi.sr")[0]
    subs =  pysrt.open(subtitles_path + filename)
    for l in subs:
        sub = {'program': program_id}
        sub['content'] = str(l.text)
        sub['start'] = str(l.start)
        sub['start_s'] = to_seconds(str(l.start))
        sub['end'] = str(l.end)
        sub['end_s'] = to_seconds(str(l.end))
        sub['duration_s'] = sub['end_s'] - sub['start_s']
        data.append(sub)

In [ ]:
df = pd.DataFrame(data)[['program', 'content', 'start', 'end', 'start_s', 'end_s', 'duration_s']]

In [ ]:
df.head()

In [ ]:
! mkdir data/yle_subtitles

In [ ]:
df.to_csv('data/yle_subtitles/urheiluruutu_subs.csv', index=False)

# Sparql Query for the segments

# Sparql Query for the programs

# Process metadata files

In [ ]:
import xml.etree.ElementTree as ET
from tqdm.notebook import tqdm

In [ ]:
data_path = '../rdf-converter/data/yle/15-urheiluruutu'
files = os.listdir(data_path)
for ii, file in enumerate(tqdm(files)):

    root = ET.parse(data_path+'/'+file).getroot()
    guid = root.find("./MAObject[1]/GUID").text

    fi_title       = root.find("./MAObject[1]/Meta/[@name='FI_TITLE']").text
    duration_tc    = root.find("./MAObject[1]/Meta/[@name='SYSTEM_DURATION_TC']").text 
    

In [ ]:
df_parts.head()

In [ ]:
df_programs = pd.read_csv('data/urheiluruutu_programs.csv')

In [ ]:
df_programs.head()

In [ ]:
df_programs['segment_id'] = df_programs['program_id']
df_programs['program'] = df_programs['program_id']

df_programs['duration_s'] = df_programs['duration']

In [ ]:
df_programs.head()

In [ ]:
df_parts['segment_id'] = df_parts['part'].map(lambda u: u.split('/')[-1])
df_parts['program'] = df_parts['program_id']

df_parts['start_s'] = df_parts['start'].map(to_seconds)
df_parts['end_s'] = df_parts['end'].map(to_seconds)
df_parts['duration_s'] = df_parts['end_s'] - df_parts['start_s']

In [ ]:
df_parts.head()

In [ ]:
df_segs = df_parts[['segment_id', 'program', 'title', 'start', 'duration', 'start_s', 'duration_s', 'end_s']]

In [ ]:
segs = pd.concat([df_segs, df_programs])[['segment_id', 'program', 'title', 'start', 'duration', 'start_s', 'duration_s', 'end_s']]

In [ ]:
segs

In [ ]:
segs.to_csv('data/yle_subtitles/urheiluruutu_segs.csv', index=False)

In [ ]:
segs[segs.segment_id.str.startswith('PROG')]